In [19]:
import pandas as pd
import librosa
import os
import numpy as np 
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

In [64]:
from keras import backend as keras_backend
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, LeakyReLU, SpatialDropout2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from keras.callbacks import ModelCheckpoint 
from keras.regularizers import l2

In [61]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

ModuleNotFoundError: No module named 'torch'

In [8]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [1]:
import tensorflow as  tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


ModuleNotFoundError: No module named 'tensorflow'

## Arquivo de metadados

In [10]:
audio_path = 'spectogram/'
spectograms = os.listdir(audio_path)

#metadata with most_freq chords indicator
df = pd.read_csv('metadata_final.csv', sep = ',')
df_chords = df[df['most_freq'] == 1]


## Mel Spectogram

In [31]:
X = np.load("X-mel_spec.npy")
y = np.load("y-mel_spec.npy")

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [51]:
le = LabelEncoder()
y_test_encoded = to_categorical(le.fit_transform(y_test))
y_train_encoded = to_categorical(le.fit_transform(y_train))

In [54]:
X_train.shape

(755, 40, 608)

In [56]:
# How data should be structured
num_rows = 608
num_columns = 40 
num_channels = 1

# Reshape to fit the network input (channel last)
X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns, num_channels)
X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

# Total number of labels to predict (equal to the network output nodes)
num_labels = y_train_encoded.shape[1]

In [65]:
def create_model():

    # Create a secquential object
    model = Sequential()


    # Conv 1
    model.add(Conv2D(filters=32, 
                     kernel_size=(3, 3), 
                     input_shape=(num_rows, num_columns, num_channels)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=32, 
                     kernel_size=(3, 3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())


    # Max Pooling #1
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, 
                     kernel_size=(3, 3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=64, 
                     kernel_size=(3,3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    
   
    # Reduces each h×w feature map to a single number by taking the average of all h,w values.
    model.add(GlobalAveragePooling2D())


    # Softmax output
    model.add(Dense(num_labels, activation='softmax'))
    
    return model


model = create_model()

In [66]:
model.compile(
    loss='binary_crossentropy',  # duas classes
    metrics=['accuracy'],  
    optimizer='adam')

# Display model architecture summary 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 606, 38, 32)       320       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 606, 38, 32)       0         
                                                                 
 batch_normalization (Batch  (None, 606, 38, 32)       128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 604, 36, 32)       9248      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 604, 36, 32)       0         
                                                                 
 batch_normalization_1 (Bat  (None, 604, 36, 32)       128       
 chNormalization)                                       

In [67]:
history = model.fit(X_train, 
                    y_train_encoded, 
                    batch_size=4, 
                    epochs=30, 
                    validation_data=(X_test,y_test_encoded),
                    verbose=1)

Epoch 1/30
189/189 [==============================] - 59s 305ms/step - loss: 0.5275 - accuracy: 0.1377 - val_loss: 0.5615 - val_accuracy: 0.0992
Epoch 2/30
189/189 [==============================] - 74s 390ms/step - loss: 0.3221 - accuracy: 0.1695 - val_loss: 0.4224 - val_accuracy: 0.1421
Epoch 3/30
189/189 [==============================] - 66s 350ms/step - loss: 0.3137 - accuracy: 0.1907 - val_loss: 0.3506 - val_accuracy: 0.1072
Epoch 4/30
189/189 [==============================] - 72s 383ms/step - loss: 0.3136 - accuracy: 0.1868 - val_loss: 0.3152 - val_accuracy: 0.1984
Epoch 5/30
189/189 [==============================] - 52s 273ms/step - loss: 0.3120 - accuracy: 0.2040 - val_loss: 0.3204 - val_accuracy: 0.1903
Epoch 6/30
189/189 [==============================] - 53s 280ms/step - loss: 0.3112 - accuracy: 0.1947 - val_loss: 0.3162 - val_accuracy: 0.1689
Epoch 7/30
189/189 [==============================] - 61s 323ms/step - loss: 0.3088 - accuracy: 0.2066 - val_loss: 0.3341 - val_ac